## Data Preparation

In [ ]:
import librosa
import os

# Load and extract MFCC features from audio files
audio_data = []
labels = []

audio_dir = "your_audio_data_directory"
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
        audio, _ = librosa.load(os.path.join(audio_dir, filename), sr=16000)  # Load audio at a specific sample rate
        mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=13)
        audio_data.append(mfccs.T)
        labels.append(0)  # Assign labels (0 for one class)


## Audio Preprocessing

In [ ]:
import numpy as np

# Preprocess and organize audio features into sequences
sequence_length = 100  # Set a sequence length
n_features = mfccs.shape[0]
n_frames = mfccs.shape[1]

audio_data_sequences = []
for mfcc in audio_data:
    for i in range(0, n_frames - sequence_length, sequence_length):
        audio_data_sequences.append(mfcc[:, i:i+sequence_length])

audio_data_sequences = np.array(audio_data_sequences)


## Importing the Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras


## Build the LSTM Model

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(sequence_length, n_features)))
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=False))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 1 for binary classification


## Compile the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## Train the Model

In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(audio_data_sequences, labels, test_size=0.2, random_state=42)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=2, validation_data=(x_test, y_test))
